# Sessie 2 – RAG Evaluatie met ragas

Evalueer een minimale RAG-pijplijn met behulp van ragas-metrics: antwoord_relevantie, betrouwbaarheid, context_nauwkeurigheid.


# Scenario
Dit scenario evalueert lokaal een minimale Retrieval Augmented Generation (RAG)-pipeline. We:
- Definiëren een kleine synthetische documentencorpus.
- Embedden documenten en implementeren een eenvoudige similariteitsretriever.
- Genereren onderbouwde antwoorden met een lokaal model (Foundry Local / OpenAI-compatibel).
- Berekenen ragas-metrics (`answer_relevancy`, `faithfulness`, `context_precision`).
- Ondersteunen een SNELLE modus (omgeving `RAG_FAST=1`) om alleen de relevantie van antwoorden te berekenen voor snelle iteratie.

Gebruik dit notebook om te valideren dat jouw lokale model + embeddings-stack feitelijk onderbouwde antwoorden produceert voordat je opschaalt naar grotere corpora.


### Uitleg: Installatie van afhankelijkheden
Installeert vereiste bibliotheken:
- `foundry-local-sdk` voor lokaal modelbeheer.
- `openai` clientinterface.
- `sentence-transformers` voor dense embeddings.
- `ragas` + `datasets` voor evaluatie en metriekberekening.
- `langchain-openai` adapter voor de ragas LLM-interface.

Veilig om opnieuw uit te voeren; sla over als de omgeving al is voorbereid.


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### Uitleg: Kernimporten & Metrieken
Laadt kernbibliotheken en ragas-metrieken. Belangrijke onderdelen:
- SentenceTransformer voor embeddings.
- `evaluate` + geselecteerde ragas-metrieken.
- `Dataset` voor het samenstellen van de evaluatiecorpus.
Deze importen veroorzaken geen externe oproepen (behalve mogelijk modelcache-lading voor embeddings).


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### Uitleg: Speelgoedcorpus & QA Grondwaarheid
Definieert een miniatuur in-memory corpus (`DOCS`), een set gebruikersvragen en verwachte grondwaarheidsantwoorden. Hiermee kun je snel en deterministisch metriekberekeningen uitvoeren zonder externe gegevens op te halen. In echte scenario's zou je productievragen en zorgvuldig samengestelde antwoorden gebruiken.


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### Uitleg: Service Init, Embeddings & Veiligheidspatch
Initialiseert Foundry Local manager, past een schema-drift veiligheidspatch toe voor `promptTemplate`, lost model-id op, maakt een OpenAI-compatibele client aan, en berekent vooraf dense embeddings voor de documentencorpus. Dit stelt herbruikbare status in voor ophalen + genereren.


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### Uitleg: Retriever-functie
Definieert een eenvoudige vector-similariteitsretriever die gebruikmaakt van de dotproduct over genormaliseerde embeddings. Geeft de top-k documenten terug (standaard k=2). Vervang in productie door een ANN-index (FAISS, Chroma, Milvus) voor schaalbaarheid en lagere latentie.


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### Uitleg: Generatiefunctie
`generate` stelt een beperkt prompt samen (systeem instrueert om ALLEEN context te gebruiken) en roept het lokale model aan. Een lage temperatuur (0.1) bevordert nauwkeurige extractie boven creativiteit. Geeft teruggesneden antwoordtekst terug.


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### Uitleg: Fallback Client Initialisatie
Zorgt ervoor dat `client` bestaat, zelfs als een eerdere initialisatiecel is overgeslagen of mislukt—voorkomt een NameError tijdens latere evaluatiestappen.


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### Uitleg: Evaluatielus & Metrieken
Stelt de evaluatiedataset samen (vereiste kolommen: vraag, antwoord, contexten, grondwaarheden, referentie) en doorloopt vervolgens de geselecteerde ragas-metrieken.

Optimalisatie:
- FAST_MODE beperkt zich tot de relevantie van antwoorden voor snelle rooktests.
- Per-metriek lus voorkomt volledige herberekening wanneer één metriek faalt.

Geeft een dict terug van metriek -> score (NaN bij falen).


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}


---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
